# Tutorial machine learning NEB algorithm (ML-NEB).

This tutorial is meant to be used to familiarized yourself with the machine learning algorithm ML-NEB.

## 0. Important notes:

When running ML-NEB, the user must feed an ASE (e.g. VASP, EMT, CASTEP, QE,...) calculator to the "MLNEB" class.
This calculator must contain the same flags as when performing a single-point calculation.

For instance, when using the Vasp calculator one must set:

    nsw=0

when using Quantum Espresso one must set:

    calculation='scf' 

when using ase-espresso:

    mode='scf'

## 1. How to run ML-NEB.

For this example we will build the Atoms structures for the initial and final end-points of our NEB calculation.

First, we will start by setting up an ASE calculator. In this case, we will use the EMT() potential as implemented in ASE:

In [ ]:
from ase.calculators.emt import EMT

ase_calculator = EMT()

We will create a 2$\times$2-Al(100) slab in which we will add an Al adatom on top of the surface:

In [ ]:
from ase.build import fcc100, add_adsorbate
from ase.constraints import FixAtoms


# 2x2-Al(001) surface with 3 layers and an
# Au atom adsorbed in a hollow site:
slab = fcc100('Al', size=(2, 2, 3))
add_adsorbate(slab, 'Au', 1.7, 'hollow')
slab.center(axis=2, vacuum=4.0)

# Fix second and third layers:
mask = [atom.tag > 1 for atom in slab]
slab.set_constraint(FixAtoms(mask=mask))

Then, we will append the ASE calculator to the Atoms object:

In [ ]:
slab.set_calculator(ase_calculator)

And we will carry out the optimization of the initial and final end-points:

In [ ]:
from ase.optimize import BFGS

# Initial end-point:
qn = BFGS(slab, trajectory='initial.traj')
qn.run(fmax=0.01)

# Final end-point:
slab[-1].x += slab.get_cell()[0, 0] / 2
qn = BFGS(slab, trajectory='final.traj')
qn.run(fmax=0.01)

The trajectory files containing the optimized structures ("initial.traj" and "final.traj") will be used in our ML-NEB calculation as the starting positions for the NEB.

You can select the number of images for the ML-NEB as an integer (e.g. 11 images) or alternately, you can automatically select the number of initial images deciding the spacing between them (in Angstrom). This is done by introducing a float (e.g. 0.2) in "n_images".
Running ML-NEB might take a few seconds...

In [ ]:
from catlearn.optimize.mlneb import MLNEB

neb_catlearn = MLNEB(start='initial.traj', # Initial end-point.
                     end='final.traj', # Final end-point.
                     ase_calc=ase_calculator, # Calculator, it must be the same as the one used for the optimizations.
                     n_images=7, # Number of images (interger or float, see above).
                     interpolation='idpp', # Choose between linear or idpp interpolation (as implemented in ASE).
                     restart=False)
neb_catlearn.run(fmax=0.05, trajectory='ML-NEB.traj')

Congratulations, your ML-NEB is converged. Now we will proceed to analyze the output of this run...

----------------------------------

------------------------------

## 2. Plot ML-NEB results.


In order to plot the predicted path we can use the results stored in the files "results_neb.csv" and "results_neb_interpolation.csv". These files contains the position of the images with respect to the length of the band and their corresponding energies and uncertainties.

To plot the converged path one can use our tool "plotneb" as:

In [ ]:
from catlearn.optimize.tools import plotneb
plotneb(trajectory='ML-NEB.traj', view_path=True)

---------------------------

----------------------------

-----------------------------------

## 3. Restarting ML-NEB.

Restarting a NEB calculation is not a tedious process anymore (WOOHOO!). 
Let's imagine that our ML-NEB has not converged. This is common ocurrence when using computer clusters and computationally demanding calculations. In this example, the calculation will not converged because the number of iterations will exceed the maximum number of steps (here maximum steps is set to 5).

In [ ]:
neb_catlearn = MLNEB(start='initial.traj', # Initial end-point.
                     end='final.traj', # Final end-point.
                     ase_calc=ase_calculator, # Calculator, it must be the same as the one used for the optimizations.
                     n_images=7, # Number of images (interger or float, see above).
                     interpolation='idpp',
                     restart=False # Choose between linear or idpp interpolation (as implemented in ASE).
                    )
neb_catlearn.run(fmax=0.05, trajectory='ML-NEB.traj', steps=5)

When using ML-NEB the most important file is the "evaluated_structures.traj" file. 
This file contains all the Atoms structures evaluated in each iteration.
In order to restart the ML-NEB calculation from the last iteration make sure you run the calculation in the same folder than the one containing the "evaluated_structures.traj".
Then, select "restart=True" in the MLNEB class:        

In [ ]:
neb_catlearn = MLNEB(start='initial.traj', # Initial end-point.
                     end='final.traj', # Final end-point.
                     ase_calc=ase_calculator, # Calculator, it must be the same as the one used for the optimizations.
                     n_images=7, # Number of images (interger or float, see above).
                     interpolation='idpp', # Choose between linear or idpp interpolation (as implemented in ASE).
                     restart=True)
neb_catlearn.run(fmax=0.05, trajectory='ML-NEB.traj')

We can also change the number of images, increase the convergence criteria and re-run (restart) the calculation.

In [ ]:
neb_catlearn = MLNEB(start='initial.traj', # Initial end-point.
                     end='final.traj', # Final end-point.
                     ase_calc=ase_calculator, # Calculator, it must be the same as the one used for the optimizations.
                     n_images=21, # Number of images (interger or float, see above).
                     interpolation='idpp', # Choose between linear or idpp interpolation (as implemented in ASE).
                     restart=True)
neb_catlearn.run(fmax=0.02, unc_convergence=0.025, trajectory='ML-NEB.traj')


In [ ]:
plotneb(trajectory='ML-NEB.traj', view_path=True)